In [1]:
import os

os.chdir("../scripts")

from data_processing import poquad, processing
from t5.load_t5 import *

In [2]:
train_df, valid_df = poquad.load_poquad_manually_downloaded("../data/poquad-manually-processed/")

In [3]:
train_input = poquad.dataset_into_str_input(train_df)

In [4]:
valid_input = poquad.dataset_into_str_input(valid_df)

In [5]:
tokenizer, model = load_plt5("../models/plt5-original-small")

In [5]:
train_input

,input_text,target_text
0,kontekst: Konfederacja polsko-czechosłowacka ...,odpowiedź: wymianą listów Ripka – Stroński
1,kontekst: Konfederacja polsko-czechosłowacka ...,odpowiedź: E. Beneša i J. Masaryka z jednej a ...
2,kontekst: Konfederacja polsko-czechosłowacka ...,odpowiedź: podpisanie układu konfederacyjnego
3,kontekst: Konfederacja polsko-czechosłowacka ...,odpowiedź: E. Beneš
4,kontekst: Konfederacja polsko-czechosłowacka ...,odpowiedź: 20 listopada 1942
...,...,...
46182,kontekst: RMS Titanic Na jaw wyszła archaiczn...,odpowiedź: wyporności
46183,kontekst: RMS Titanic Na jaw wyszła archaiczn...,odpowiedź: statek pełnomorski o wyporności pow...
46184,kontekst: RMS Titanic Na jaw wyszła archaiczn...,odpowiedź: „Titanic” spełniał te wymagania z d...
46185,kontekst: RMS Titanic Na jaw wyszła archaiczn...,odpowiedź: rozbieżności między przepisem skons...


In [16]:
train_input["input_text"].sample(100)

15617    kontekst: Christina Aguilera  W grudniu 1999 j...
15011    kontekst: HCP Puma  Wózek napędny typu 8NN jes...
30631    kontekst: Sylwester (Czetwertyński)  W związku...
43484    kontekst: Wróbel krótkopalcowy  Jedynym podobn...
1124     kontekst: Jacek Laskowski  W latach 1991–1998 ...
                               ...                        
42773    kontekst: Pesa 301Dd  Podczas modernizacji zas...
27233    kontekst: Solaris Urbino 12  Solaris Urbino 12...
2243     kontekst: Znachor (film 1981)  Z perspektywy c...
23451    kontekst: Rezerwat przyrody Hedera  Rezerwat u...
27837    kontekst: Jay Sebring  Został zamordowany prze...
Name: input_text, Length: 100, dtype: object

In [69]:
sample = train_input["target_text"].sample(100)

max_size = 0
for i in range(len(sample)):
    size = tokenizer(sample.iloc[i], return_tensors="pt").input_ids.shape[1]
    if size > max_size:
        max_size = size
        print(max_size)

8
11
18
20
21
23
26
30
69


In [9]:
train_df.head(10)

,title,context,question,is_impossible,answers
1,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Co było powodem powrócenia konceptu porozumien...,False,"{'answer_start': [117], 'text': ['wymianą list..."
2,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Pomiędzy jakimi stronami odbyło się zgromadzen...,False,"{'answer_start': [197], 'text': ['E. Beneša i ..."
3,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,O co ubiegali się polscy przedstawiciele podcz...,False,"{'answer_start': [315], 'text': ['podpisanie u..."
4,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Który z dyplomatów sprzeciwił się konceptowi k...,False,"{'answer_start': [558], 'text': ['E. Beneš']}"
5,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Kiedy oficjalnie doszło do zawarcia porozumienia?,False,"{'answer_start': [691], 'text': ['20 listopada..."
6,Pomilio PD/PE,Samoloty Pomilio PD weszły na wyposażenie jedn...,Jaki model zastąpił Pomilio PC?,False,"{'answer_start': [9], 'text': ['Pomilio PD']}"
7,Pomilio PD/PE,Samoloty Pomilio PD weszły na wyposażenie jedn...,Co wchodziło w skład eskadry 139?,False,"{'answer_start': [391], 'text': ['dwóch sekcji..."
8,Pomilio PD/PE,Samoloty Pomilio PD weszły na wyposażenie jedn...,Jaki model zastąpił samoloty Pomilio PD?,False,"{'answer_start': [683], 'text': ['PE']}"
9,Pomilio PD/PE,"Na początku 1918 roku planowano, że Pomilio PD...",Czym według założeń z 1918 roku miały zajmować...,False,"{'answer_start': [60], 'text': ['wyposażeniem ..."
10,Pomilio PD/PE,"Na początku 1918 roku planowano, że Pomilio PD...",Jaki samolot miał służyć za podstawę rozpoznan...,False,"{'answer_start': [195], 'text': ['Ansaldo SIA ..."


In [7]:
import torch
from typing import List, Dict, Any
from dataclasses import dataclass
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [8]:
@dataclass
class T2TDataCollator(DataCollatorWithPadding):
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['target_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])


        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'target_ids': lm_labels,
            'target_attention_mask': decoder_attention_mask
        }


In [12]:
class T2TDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        # Assuming 'batch' is a list of examples, where each example is a dictionary
        # that includes 'input_ids' and 'labels' (target sequence) among other possible keys.
        input_ids = [example['input_ids'] for example in batch]
        labels = [example['labels'] for example in batch]
        
        # Prepare decoder_input_ids
        decoder_input_ids = [self._shift_right(label) for label in labels]
        
        # Tokenizer's pad method can handle padding of both input_ids and decoder_input_ids
        batch = self.tokenizer.pad(
            {"input_ids": input_ids, "decoder_input_ids": decoder_input_ids},
            return_tensors="pt"
        )
        
        # Ensure labels are also included and properly padded
        batch["labels"] = self.tokenizer.pad(
            {"input_ids": labels},
            return_tensors="pt"
        )["input_ids"]
        
        return batch

    def _shift_right(self, labels):
        # Shift the labels to the right to create decoder_input_ids
        return [self.tokenizer.pad_token_id] + labels[:-1]

In [8]:
import wandb

In [9]:
wandb.init(
    # set the wandb project where this run will be logged
    project="PLT5 Small Finetuning Poquad",
    # track hyperparameters and run metadata
    # config={
    # "architecture": "PLT5 Small",
    # "dataset": "Poquad",
    # }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: m-tkacz6 (m-tkacz-uw). Use `wandb login --relogin` to force relogin


In [10]:
from transformers import TrainerCallback

class PrintMemoryUsageCallback(TrainerCallback):
    """ Callback that prints memory allocation during training """
    def on_step_end(self, args, state, control, **kwargs):
        print(f"Step {state.global_step}: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB allocated")

In [13]:
train_dataset = processing.TextDataset(train_input, tokenizer, 1024, 128)
valid_dataset = processing.TextDataset(valid_input, tokenizer, 1024, 128)

# Create DataCollator
data_collator = T2TDataCollator(tokenizer)

# Initialize model

# Define TrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=1,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=4,
    logging_dir='./logs',
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        callbacks=[PrintMemoryUsageCallback()],
        )

# Train the model
trainer.train()

# Save the model
trainer.save_model('../models/plt5-small-1epoch')

print("Training complete and model saved to ./trained_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/2886 [00:00<?, ?it/s]

Step 1: 380.41 MB allocated
Step 2: 380.41 MB allocated
Step 3: 380.41 MB allocated
Step 4: 380.41 MB allocated
Step 5: 380.41 MB allocated
Step 6: 380.41 MB allocated
Step 7: 380.41 MB allocated
Step 8: 380.41 MB allocated
Step 9: 380.41 MB allocated
Step 10: 380.41 MB allocated
Step 11: 380.41 MB allocated
Step 12: 380.41 MB allocated
Step 13: 380.41 MB allocated
Step 14: 380.41 MB allocated
Step 15: 380.41 MB allocated
Step 16: 380.41 MB allocated
Step 17: 380.41 MB allocated
Step 18: 380.41 MB allocated
Step 19: 380.41 MB allocated
Step 20: 380.41 MB allocated
Step 21: 380.41 MB allocated
Step 22: 380.41 MB allocated
Step 23: 380.41 MB allocated
Step 24: 380.41 MB allocated
Step 25: 380.41 MB allocated
Step 26: 380.41 MB allocated
Step 27: 380.41 MB allocated
Step 28: 380.41 MB allocated
Step 29: 380.41 MB allocated
Step 30: 380.41 MB allocated
Step 31: 380.41 MB allocated
Step 32: 380.41 MB allocated
Step 33: 380.41 MB allocated
Step 34: 380.41 MB allocated
Step 35: 380.41 MB allo